In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch.utils.data as data


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
import ast
# Đọc dữ liệu từ file CSV
df = pd.read_csv('data.csv')
    
# Chọn các cột 'x1', 'x2', 'x3', 'x4', 'x5' làm đầu vào và 'y' làm đầu ra
x_data = df[['x1', 'x2', 'x3', 'x4', 'x5']]
y_data = df['y']

# Khởi tạo danh sách để lưu trữ dữ liệu đầu vào dưới dạng tensors
x_train = []

# Lặp qua từng hàng trong x_data
for index in range(len(x_data)):
    # Lấy hàng thứ index từ DataFrame
    row = x_data.iloc[index]
    # Chuyển đổi từng giá trị trong hàng thành list
    x_values = [ast.literal_eval(val) for val in row.values]
    # Thêm tensor tương ứng vào danh sách x_train
    x_train.append(torch.tensor(x_values, dtype=torch.float32))

# Chuyển đổi dữ liệu đầu ra thành tensor PyTorch
y_train = torch.tensor(y_data.values, dtype=torch.long)

# Tạo tensor từ danh sách các tensors đầu vào
X_train = torch.stack(x_train)

# In ra kích thước của tensors đầu vào và đầu ra
print("x_train:", X_train.shape)
print("y_train:", y_train.shape)

x_train: torch.Size([26846, 5, 10])
y_train: torch.Size([26846])


In [4]:
batch_size = 32
length = 10
num_labels = 36
num_sensors = 5

In [5]:
# Tổng số mẫu
total_samples = len(x_train)

# Tính số lượng mẫu cho tập huấn luyện và tập kiểm tra
train_size = int(0.8 * total_samples)  # 80% cho tập huấn luyện
test_size = total_samples - train_size  # 20% cho tập kiểm tra

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
train_dataset, test_dataset = data.random_split(torch.utils.data.TensorDataset(X_train, y_train),
                                                [train_size, test_size])

# Khởi tạo DataLoader cho cả tập huấn luyện và tập kiểm tra
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

### CNN model

In [6]:
# Định nghĩa mô hình CNN 1D
class CNN1D(nn.Module):
    def __init__(self, input_size, num_classes):
        super(CNN1D, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=input_size, out_channels=128, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3, padding=1)
        self.conv3 = nn.Conv1d(in_channels=256, out_channels=512, kernel_size=3, padding=1)
        self.conv4 = nn.Conv1d(in_channels=512, out_channels=512, kernel_size=3, padding=1)
        self.conv5 = nn.Conv1d(in_channels=512, out_channels=1024, kernel_size=3, padding=1)
        self.dropout = nn.Dropout(0.3)
        self.maxpool = nn.MaxPool1d(kernel_size=2, stride=1)
        self.fc = nn.Linear(1024 * length, num_labels)    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.maxpool(x)
        x = F.relu(self.conv2(x))
        x = self.maxpool(x)
        x = F.relu(self.conv3(x))
        x = self.maxpool(x)
        x = F.relu(self.conv4(x))
        x = self.maxpool(x)
        x = F.relu(self.conv5(x))
        x = self.maxpool(x)
        x = x.view(x.size(0), -1)  # Flatten the tensor for the fully connected layer
        x = self.dropout(x)
        x = self.fc(x)
        return x

In [7]:
# Huấn luyện mô hình
num_epochs = 100
batch_size = 32
learning_rate = 0.0001
weight_decay = 0.0001  # L2 regularization

In [8]:
# Khởi tạo mô hình
input_size = num_sensors  # Số lượng cảm biến
model = CNN1D(input_size=input_size, num_classes=num_labels).to(device)

In [9]:
# Khởi tạo optimizer và loss function
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

In [10]:
# Tạo label map từ chuỗi nhãn sang chỉ mục nguyên
label_map = {}
index = 0
for i in range(1, 10):
    for j in range(1, 5):
        # Biểu diễn vị trí của mỗi ô trong lưới 9x4 dưới dạng chuỗi
        label = f"{i}{j}"
        # Ánh xạ chuỗi nhãn sang chỉ mục nguyên
        label_map[label] = index
        index += 1

# In label map để kiểm tra
print(label_map)


# Hàm chuyển đổi chuỗi nhãn thành chỉ mục nguyên
def label_to_index(labels):
    # Chuyển đổi tensor labels về list giá trị của nó
    labels_list = [str(label.item()) for label in labels]
    # Sử dụng list giá trị để truy cập từ điển ánh xạ
    return torch.tensor([label_map[label] for label in labels_list])

# Tạo từ điển ánh xạ từ chỉ mục lớp sang nhãn
index_to_label = {index: label for label, index in label_map.items()}

# Hàm chuyển đổi chỉ mục thành nhãn
def index_to_label_func(index):
    return index_to_label[index]


{'11': 0, '12': 1, '13': 2, '14': 3, '21': 4, '22': 5, '23': 6, '24': 7, '31': 8, '32': 9, '33': 10, '34': 11, '41': 12, '42': 13, '43': 14, '44': 15, '51': 16, '52': 17, '53': 18, '54': 19, '61': 20, '62': 21, '63': 22, '64': 23, '71': 24, '72': 25, '73': 26, '74': 27, '81': 28, '82': 29, '83': 30, '84': 31, '91': 32, '92': 33, '93': 34, '94': 35}


In [11]:
train_losses=[]
# Huấn luyện mô hình
for epoch in range(num_epochs):
    model.train()  # Chuyển sang chế độ huấn luyện
    running_loss = 0.0
    for inputs, labels_str in train_loader:
        inputs, labels_str = inputs.to(device), labels_str

        # Chuyển đổi nhãn thành chỉ mục nguyên
        labels = label_to_index(labels_str).to(device)

        # Feedforward
        outputs = model(inputs)

        # Tính loss và backward
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Lưu trữ loss cho epoch hiện tại
    train_losses.append(running_loss / (num_samples / batch_size))

    if (epoch+1)%10==0:
      # Lưu trạng thái của mô hình vào một tệp tin
      torch.save(model.state_dict(), model_file_path)
      print(f'Da luu model: {model_file_path}')
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

print('Training finished!')


RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x5120 and 10240x36)